In [131]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
from avcv.all import *

In [133]:
import copy
import os
from time import perf_counter

import click
import imageio
import numpy as np
import PIL.Image
import torch
import torch.nn.functional as F

import dnnlib
import legacy
# Load VGG16 feature detector.
url = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metrics/vgg16.pt'
device = 'cuda:0'
with dnnlib.util.open_url(url) as f:
    vgg16 = torch.jit.load(f).eval().to(device)

    
def project(
    G,
    target: torch.Tensor, # [C,H,W] and dynamic range [0,255], W & H must match G output resolution
    *,
    num_steps                  = 1000,
    w_avg_samples              = 10000,
    initial_learning_rate      = 0.1,
    initial_noise_factor       = 0.05,
    lr_rampdown_length         = 0.25,
    lr_rampup_length           = 0.05,
    noise_ramp_length          = 0.75,
    regularize_noise_weight    = 1e5,
    verbose                    = False,
    extract_fn = None,
    device: torch.device
):
    assert target.shape == (G.img_channels, G.img_resolution, G.img_resolution)

    def logprint(*args):
        if verbose:
            print(*args)

    G = copy.deepcopy(G).eval().requires_grad_(False).to(device) # type: ignore

    # Compute w stats.
    logprint(f'Computing W midpoint and stddev using {w_avg_samples} samples...')
    z_samples = np.random.RandomState(123).randn(w_avg_samples, G.z_dim)
    w_samples = G.mapping(torch.from_numpy(z_samples).to(device), None)  # [N, L, C]
    w_samples = w_samples[:, :1, :].cpu().numpy().astype(np.float32)       # [N, 1, C]
    w_avg = np.mean(w_samples, axis=0, keepdims=True)      # [1, 1, C]
    w_std = (np.sum((w_samples - w_avg) ** 2) / w_avg_samples) ** 0.5

    # Setup noise inputs.
    noise_bufs = { name: buf for (name, buf) in G.synthesis.named_buffers() if 'noise_const' in name }

    # Load VGG16 feature detector.
    # url = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metrics/vgg16.pt'

    # Features for target image.
    target_images = target.unsqueeze(0).to(device).to(torch.float32)
    if target_images.shape[2] > 256:
        target_images = F.interpolate(target_images, size=(256, 256), mode='area')
    # target_features = vgg16(target_images, resize_images=False, return_lpips=True)
    target_features = extract_fn(target_images)#, resize_images=False, return_lpips=True)

    w_opt = torch.tensor(w_avg, dtype=torch.float32, device=device, requires_grad=True) # pylint: disable=not-callable
    w_out = torch.zeros([num_steps] + list(w_opt.shape[1:]), dtype=torch.float32, device=device)
    optimizer = torch.optim.Adam([w_opt] + list(noise_bufs.values()), betas=(0.9, 0.999), lr=initial_learning_rate)

    # Init noise.
    for buf in noise_bufs.values():
        buf[:] = torch.randn_like(buf)
        buf.requires_grad = True
        
    pbar = tqdm(range(num_steps))
    for step in pbar:
        # Learning rate schedule.
        t = step / num_steps
        w_noise_scale = w_std * initial_noise_factor * max(0.0, 1.0 - t / noise_ramp_length) ** 2
        lr_ramp = min(1.0, (1.0 - t) / lr_rampdown_length)
        lr_ramp = 0.5 - 0.5 * np.cos(lr_ramp * np.pi)
        lr_ramp = lr_ramp * min(1.0, t / lr_rampup_length)
        lr = initial_learning_rate * lr_ramp
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # Synth images from opt_w.
        w_noise = torch.randn_like(w_opt) * w_noise_scale
        ws = (w_opt + w_noise).repeat([1, G.mapping.num_ws, 1])
        synth_images = G.synthesis(ws, noise_mode='const')

        # Downsample image to 256x256 if it's larger than that. VGG was built for 224x224 images.
        synth_images = (synth_images + 1) * (255/2)
        if synth_images.shape[2] > 256:
            synth_images = F.interpolate(synth_images, size=(256, 256), mode='area')

        # Features for synth images.
        # synth_features = vgg16(synth_images, resize_images=False, return_lpips=True)
        synth_features = extract_fn(synth_images)#, resize_images=False, return_lpips=True)
        dist = (target_features - synth_features).square().sum()

        # Noise regularization.
        reg_loss = 0.0
        for v in noise_bufs.values():
            noise = v[None,None,:,:] # must be [1,1,H,W] for F.avg_pool2d()
            while True:
                reg_loss += (noise*torch.roll(noise, shifts=1, dims=3)).mean()**2
                reg_loss += (noise*torch.roll(noise, shifts=1, dims=2)).mean()**2
                if noise.shape[2] <= 8:
                    break
                noise = F.avg_pool2d(noise, kernel_size=2)
        loss = dist + reg_loss * regularize_noise_weight

        # Step
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        pbar.set_description(f'step {step+1:>4d}/{num_steps}: dist {dist:<4.2f} loss {float(loss):<5.2f}')

        # Save projected W for each optimization step.
        w_out[step] = w_opt.detach()[0]

        # Normalize noise.
        with torch.no_grad():
            for buf in noise_bufs.values():
                buf -= buf.mean()
                buf *= buf.square().mean().rsqrt()

    return w_out.repeat([1, G.mapping.num_ws, 1])


In [123]:
network_pkl = 'training-runs/00006-stylegan2-cropfaces-gpus6-batch24-gamma6.6/network-snapshot-004869.pkl'
# target_fname = '/shared/RLDD/Fold1_part1_05_10/face_only/images/000052.jpg'
target_fname = ''
outdir='results/projected_517'
save_video=True
seed=0
device = 'cuda:0'

In [23]:
import sys
sys.path.insert(0, '/home/anhvth8/YOLOX/lit_classifier/eyestate_prediction/')

In [24]:
# %cd /home/anhvth8/YOLOX/lit_classifier/eyestate_prediction/
from pretrained_networks import pretrained_factory
# if not 'clip' in dir():
# clip = pretrained_factory('clip').to(device)

In [104]:
import clip
model = clip.load("ViT-B/32", device=device)[0]
model = model.requires_grad_(False)

In [72]:
import timm
model = timm.create_model('resnet50', pretrained=True).to(device).eval().requires_grad_(False)

In [117]:
# preproc(torch)
from torchvision import transforms
normalize = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

In [118]:

def extract_feature_clip(inputs):
    inputs = F.interpolate(inputs.to(device), size=(224,224))
    inputs = inputs/255
    inputs = normalize(inputs)
    return model.encode_image(inputs)

def extract_feature_vgg(inputs):
    target_features = vgg16(inputs, resize_images=False, return_lpips=True)
    return target_features

In [124]:
num_steps=256

In [129]:
np.random.seed(seed)
torch.manual_seed(seed)

# Load networks.
print('Loading networks from "%s"...' % network_pkl)
if not 'G' in dir():
    with dnnlib.util.open_url(network_pkl) as fp:
        G = legacy.load_network_pkl(fp)['G_ema'].requires_grad_(False).to(device) # type: ignore

# Load target image.
# target_pil = PIL.Image.open(target_fname).convert('RGB')
# target_pil = PIL.Image.open(target_fname).convert('GRAY')

targets = []
target_fnames = list(sorted(glob(os.path.join(input_dir, '*'))))[:1]
target_uint8s = []
# for target_fname in target_fnames:
# img = cv2.imread(target_fname, 0)
# img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
img = mmcv.imread(target_fname, channel_order='rgb')
img = mmcv.imrescale(img, (256,256))
img = mmcv.impad(img, shape=(256,256))

target_pil = PIL.Image.fromarray(img)
w, h = target_pil.size
s = min(w, h)
target_pil = target_pil.crop(((w - s) // 2, (h - s) // 2, (w + s) // 2, (h + s) // 2))
target_pil = target_pil.resize((G.img_resolution, G.img_resolution), PIL.Image.LANCZOS)
target_uint8 = np.array(target_pil, dtype=np.uint8)
targets.append(torch.tensor(target_uint8.transpose([2, 0, 1]), device=device))
target_uint8s.append(target_uint8)
    
    
# targets = torch.stack(targets)

# Optimize projection.
start_time = perf_counter()
projected_w_steps = project(
    G,
    target=targets[0], # pylint: disable=not-callable
    num_steps=num_steps,
    device=device,
    verbose=True,
    extract_fn=extract_feature_vgg,
)

print (f'Elapsed: {(perf_counter()-start_time):.1f} s')

Loading networks from "training-runs/00006-stylegan2-cropfaces-gpus6-batch24-gamma6.6/network-snapshot-003346.pkl"...
Computing W midpoint and stddev using 10000 samples...


step  256/256: dist 0.16 loss 0.16 : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:20<00:00, 12.21it/s]

Elapsed: 21.2 s


In [130]:
# Render debug output: optional video and projected image and W vector.
os.makedirs(outdir, exist_ok=True)
if save_video:
    video = imageio.get_writer(f'{outdir}/proj.mp4', mode='I', fps=10, codec='libx264', bitrate='16M')
    out_video_path = osp.abspath(f"{outdir}/proj.mp4")
    print (f'Saving optimization progress video {out_video_path}')
    for projected_w in projected_w_steps:
        synth_image = G.synthesis(projected_w.unsqueeze(0), noise_mode='const')
        synth_image = (synth_image + 1) * (255/2)
        synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
        video.append_data(np.concatenate([target_uint8s[0], synth_image], axis=1))
    video.close()

# Save final projected frame and W vector.
target_pil.save(f'{outdir}/target.png')
projected_w = projected_w_steps[-1]
synth_image = G.synthesis(projected_w.unsqueeze(0), noise_mode='const')
synth_image = (synth_image + 1) * (255/2)
synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
PIL.Image.fromarray(synth_image, 'RGB').save(f'{outdir}/proj.png')
np.savez(f'{outdir}/projected_w.npz', w=projected_w.unsqueeze(0).cpu().numpy())

Saving optimization progress video /home/anhvth8/gitprojects/stylegan3/results/projected_517/proj.mp4
